Goal: learning from names in names.txt, and creating more names.

This is a 'character-level' language model, where we try to predict the next character.


These are examples of language models:

Bigram (one character predicts the next one with a lookup table of counts)

MLP, following Bengio et al. 2003

CNN, following DeepMind WaveNet 2016 (in progress...)

RNN, following Mikolov et al. 2010

LSTM, following Graves et al. 2014

GRU, following Kyunghyun Cho et al. 2014

Transformer, following Vaswani et al. 2017

In [12]:
names = open('names.txt', 'r').read().splitlines()

In [13]:
len(names), names[:10]

(32033,
 ['emma',
  'olivia',
  'ava',
  'isabella',
  'sophia',
  'charlotte',
  'mia',
  'amelia',
  'harper',
  'evelyn'])

In [14]:
min(names, key=lambda x: len(x)), max(names, key=lambda x: len(x))

('an', 'muhammadibrahim')

# Biagram

Idea, we use i-th character, to predict (i + 1)th

Essentially, no info about [0, i-1]

In [15]:
# special tokens
START_CH = '<S>' # this means the name started
END_CH   = '<E>' # this means the name started


In [16]:
# a basic implementation with python dictionaries
# we will look at the frequency of each paris

biagram_dict = {}
for n in names:
    chs = [START_CH] + list(n) + [END_CH]
    for ch1, ch2 in zip(chs, chs[1:]):
        biagram = (ch1, ch2)
        biagram_dict[biagram] = biagram_dict.get(biagram, 0) + 1

# sort dict by value
sorted(biagram_dict.items(), key=lambda item: -item[1])

[(('n', '<E>'), 6763),
 (('a', '<E>'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', '<E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S>', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '<E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S>', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', '<E>'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('<S>', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', '<E>'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('<S>', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('<S>', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('<S>', 'l'), 1572),
 (('<S>', 'c'), 1542),
 (('<S>', 'e'), 1531),
 (('j', 'a'), 1473),
 (

In [17]:
# we can use pytorch API for implementation as well
# idea: each element of a matrix will show the count for a given row and column

import torch

In [18]:
size_of_alphabet = 26 + 2 # 2 for start and end
N = torch.zeros((size_of_alphabet, size_of_alphabet), dtype=torch.int32)

# creating a lookup table
chars = [START_CH] + sorted(list(set(''.join(names)))) + [END_CH]
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

stoi

{'<S>': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '<E>': 27}

In [224]:
# filling matrix based on frequency

for n in names:
    chs = [START_CH] + list(n) + [END_CH]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

# now that we have this matrix, we just have to sample from it.
# randomly start a new character, and randomly pick the next one based on probabilities in this matric, etc

# creating a probability matrix
# in practice, it's recommended to add a small number (like 1) to avoid zeros.
P = (N + 1).float()
P /= P.sum(1, keepdim=True)
P.shape

torch.Size([28, 28])

In [225]:
P[0,:], P[0,:].sum()

(tensor([5.2019e-06, 1.3765e-01, 4.0767e-02, 4.8138e-02, 5.2757e-02, 4.7795e-02,
         1.3025e-02, 2.0885e-02, 2.7284e-02, 1.8451e-02, 7.5604e-02, 9.2484e-02,
         4.9074e-02, 7.9224e-02, 3.5773e-02, 1.2308e-02, 1.6079e-02, 2.8766e-03,
         5.1165e-02, 6.4149e-02, 4.0835e-02, 2.4397e-03, 1.1741e-02, 9.5870e-03,
         4.1875e-03, 1.6708e-02, 2.9000e-02, 5.2019e-06]),
 tensor(1.))

In [226]:
# word creation

g = torch.Generator().manual_seed(2147483647)

def get_new_names(n):
    names = []
    for _ in range(n):
        new_name = [START_CH]
        while True:
            idx1 = stoi[new_name[-1]] # index of the last character
            p = P[idx1,:]
            idx2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
            new_ch = itos[idx2]
            if new_ch == END_CH:
                break
            new_name.append(new_ch)
        names.append(''.join(new_name[1:]))
    return names

get_new_names(10) # really terrible algorithm

['junidedianasa',
 'jush',
 'ay',
 'a',
 'nn',
 'kohin',
 'toli',
 'asate',
 'madahn',
 'auran']

In [ ]:
# example of likelihood, log likelihood, and nll

loglikelihood = 0.0
items = 0
for n in ['aliqj']:
    chs = [START_CH] + list(n) + [END_CH]
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1, idx2 = stoi[ch1], stoi[ch2]
        prob = P[idx1, idx2]
        loglikelihood += torch.log(prob)
        items += 1
        print(f"token={ch1, ch2}, {prob=:.3f}")

print(f"{loglikelihood=},  nll={-loglikelihood}, lossfunc={-loglikelihood / items}")

# Problem formulation

In [179]:
# now, we can formulate our problem like an optimization one: 
# (1) data
# (2) parameters
# (3) loss function

# data is obvious: list of names
# parameters are elements in probability matrix, P
# the loss function to use here is nll (negative log likelihood)

In [1]:
# theoretically, we could find P(i, j) by minimizing loss function using a minimizer, such as neural net.
# model:
#   input: takes a character as input (first character)
#   output: probability of having a specific character next
# we will build 27 of these models so that for a given input, it a list of probabilities for all characters
# example:
# xs = ['a']
# ys = [0.1, 0.3, ..., 0.5]
        

In [116]:
# step 1: craeting training set of biagrams (x, y)
# requires encoding

xs, ys = [], []

for n in names:
    chs = [START_CH] + list(n) + [END_CH]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)


In [118]:
xs.shape, ys.shape

(torch.Size([228146]), torch.Size([228146]))

In [119]:
# step 2: one-hot encoding.
# xs integer values are not meaningful.
# we need to transform it to dummy variables (aka one-hot encoding).
# always make sure that inputs are float

xenc = torch.nn.functional.one_hot(xs, num_classes=len(stoi)).float()
xenc.shape, xenc


(torch.Size([228146, 28]),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [121]:
# step 3: initializing weight tensor
g = torch.Generator().manual_seed(2147483647)

W = torch.rand(len(stoi), len(stoi), requires_grad=True, generator=g) # there are 28 inputs, and 28 models (output neurons) that generates 28 prob.
W.shape, W[0:5, 0:5]

# each column contains weight for each of 28 models
# xenc @ W[:,1] -> prob(second character = a | first character)
# note: this is the most simple NN ever: no hidden layer.

(torch.Size([28, 28]),
 tensor([[0.7081, 0.3542, 0.1054, 0.5996, 0.0904],
         [0.8581, 0.3224, 0.5998, 0.1621, 0.3729],
         [0.2275, 0.4577, 0.7701, 0.8770, 0.0389],
         [0.5802, 0.7099, 0.5512, 0.6437, 0.6311],
         [0.1589, 0.9281, 0.2298, 0.8939, 0.9322]], grad_fn=<SliceBackward0>))

In [133]:
for iteration in range(200):

    # step 4: forward propagation
    logits = xenc @ W # we assume these are log of counts (softmax)
    counts = logits.exp() # converting log count to count
    probs = counts / counts.sum(dim=1, keepdim=True) # normalization - shape is (#obs., 28)

    # step 5: loss function calculation
    lossfunc = -probs[torch.arange(len(probs)), ys].log().mean()
    # lossfunc = -probs[torch.arange(len(probs)), ys].log().mean() + lambda * (W ** 2).mean()
    print(f"iter={iteration} | loss={lossfunc}")

    # step 6: backward propagation
    W.grad = None # set grad zero
    lossfunc.backward()

    # step 7: update parameters
    learning_rate = 10
    W.data -= learning_rate * W.grad

# we expect the loss converge to what we saw (2.47)

iter=0 | loss=2.47235107421875
iter=1 | loss=2.472313404083252
iter=2 | loss=2.472275972366333
iter=3 | loss=2.472238779067993
iter=4 | loss=2.4722018241882324
iter=5 | loss=2.4721648693084717
iter=6 | loss=2.47212815284729
iter=7 | loss=2.4720914363861084
iter=8 | loss=2.472054958343506
iter=9 | loss=2.4720184803009033
iter=10 | loss=2.47198224067688
iter=11 | loss=2.4719462394714355
iter=12 | loss=2.471909999847412
iter=13 | loss=2.471874475479126
iter=14 | loss=2.4718387126922607
iter=15 | loss=2.4718031883239746
iter=16 | loss=2.4717679023742676
iter=17 | loss=2.4717323780059814
iter=18 | loss=2.4716973304748535
iter=19 | loss=2.4716622829437256
iter=20 | loss=2.4716274738311768
iter=21 | loss=2.471592664718628
iter=22 | loss=2.4715583324432373
iter=23 | loss=2.4715237617492676
iter=24 | loss=2.471489191055298
iter=25 | loss=2.4714553356170654
iter=26 | loss=2.4714207649230957
iter=27 | loss=2.4713869094848633
iter=28 | loss=2.47135329246521
iter=29 | loss=2.4713196754455566
iter=3

In [145]:
new_name = [START_CH]
idx1 = torch.tensor(stoi[new_name[-1]]) # index of the last character
xenc1 = torch.nn.functional.one_hot(idx1, num_classes=len(stoi)).float()
logit = xenc1 @ W
count = logit.exp()
p = count / count.sum()
idx2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
new_ch = itos[idx2]
new_ch

'd'

In [150]:
count

tensor([ 0.0733, 11.5699,  3.4218,  4.0414,  4.4299,  4.0125,  1.0877,  1.7495,
         2.2878,  1.5447,  6.3514,  7.7715,  4.1201,  6.6559,  3.0018,  1.0273,
         1.3451,  0.2444,  4.2960,  5.3880,  3.4271,  0.2103,  0.9800,  0.7986,
         0.3469,  1.3976,  2.4321,  0.0730], grad_fn=<ExpBackward0>)

In [154]:
# word creation

g = torch.Generator().manual_seed(2147483647)

def get_new_names_with_nn(n):
    names = []
    for _ in range(n):
        new_name = [START_CH]
        while True:
            idx1 = torch.tensor(stoi[new_name[-1]]) # index of the last character
            xenc1 = torch.nn.functional.one_hot(idx1, num_classes=len(stoi)).float()
            logit = xenc1 @ W
            count = logit.exp()
            p = count / count.sum()
            idx2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
            new_ch = itos[idx2]
            if new_ch == END_CH:
                break
            new_name.append(new_ch)
        names.append(''.join(new_name[1:]))
    return names

get_new_names_with_nn(10) # really terrible algorithm

['junidedianasa',
 'jush',
 'ay',
 'a',
 'nn',
 'kohin',
 'toli',
 'asate',
 'madahn',
 'auran']

In [111]:
# nll calculation concept

# nlls = torch.zeros(len(probs))

# for i in range(len(probs)):
#     # i-th biagram
#     x = xs[i].item()
#     y = ys[i].item()
#     predicted_p = probs[i, y]
#     nlls[i] = -torch.log(predicted_p)

# print(f"nll = {nlls.mean().item()}")